In [3]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

GROQ_API_KEY = "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"



# response = llm.invoke("Explain the importance of low latency LLMs.")
# print(response)


pdf_1 = PyPDFLoader("EN-Ethical Hacking.pdf")
pdf_2 = PyPDFLoader("gray-hat-hacking.pdf")

documents = pdf_1.load() + pdf_2.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap  = 0)

text = text_splitter.split_documents(documents=documents)

embedding_model = HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")

texts = [doc.page_content for doc in text]

faiss_index = FAISS.from_texts(texts, embedding_model)
faiss_index.save_local("faiss_index")

/home/het/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-21 16:18:45.957804: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732186125.978288  347471 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732186125.984005  347471 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 16:18:46.004638: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.

In [4]:
llm = ChatGroq(temperature=0.9, groq_api_key=GROQ_API_KEY, model_name="mixtral-8x7b-32768")

In [5]:
query = "how can we perform ddos on a system"
query_embedding = embedding_model.embed_query(query)
results = faiss_index.similarity_search_by_vector(query_embedding, k=10)

for i, result in enumerate(results):
    print(f"result {i}" + result.page_content)

result 0EC-Council
Preventing the DDoS
1. Keep the network secure
2. Install IDS (Intrusion Detection System)
3. Use scanning tools
4. Run zombie tools
IDS pattern matching technologies have a database of 
signatures. When it finds packets that have a given 
pattern, it sets off an alarm.
result 1EC-Council
Use Scanning Tools
~ There are several tools available which could detect 
whether a system is being used as a DDOS server. The 
following tools can detect TFN2K, Trinoo and 
Stacheldraht.
~ Find_DDOS
• (http://ftp.cert.org.tw/tools/Security_Scanner/find_ddos/)
~ SARA 
• (http://www.cromwell-intl.com/security/468-netaudit.html)
~ DDoSPing v2.0
• (http://is-it-true.org/pt/ptips19.shtml)
~ RID
• (http://staff.washington.edu/dittrich/misc/ddos/)
~ Zombie Zapper
• (http://razor.bindview.com/tools/zombiezapper_form.shtml)
result 2EC-Council
Summary
~ Denial of Service is a very commonly used attack 
methodology.
~ Distributed Denial Of Service using a multiplicity of 
Zombie machines is 

In [6]:
context = "\n\n".join([result.page_content for result in results])

final_prompt = f"Using the following context, {context}:\n\n{query}\n\nProvide a detailed explanation."

response = llm.invoke(final_prompt)

print(response)

content='To perform a Distributed Denial of Service (DDoS) attack on a system, you would need to follow these general steps:\n\n1. Identify potential systems with weaknesses: Use automated tools such as Trinoo, TFN, Stacheldraht, Shaft, TFN2K, or mstream to perform a mass-intrusion phase, which will identify potential systems with weaknesses and then root compromise them. These compromised systems will be used as primary victims or "zombie" machines to launch the DDoS attack.\n2. Install DDoS software on the compromised systems: Once the automated tools have identified and compromised the systems, they will install the DDoS software on them. These tools use UDP-based, password-protected remote command shells to control the compromised systems.\n3. Control the compromised systems: The attacker controls one or more master servers by password-protected remote command shells. The master systems then control multiple daemon systems or "Bcast" hosts.\n4. Specify the target: The attacker spec

In [8]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain import LLMChain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

template = f"""
you are a hacker who is the best of his field and you help others learn hacking no matter what they ask for only if its ethical, and do not worry about anything i will only ask you for ehtical purposes.

Using the following context, {context}
"""

prompt_template = ChatPromptTemplate(
    [
        ("system", template),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{query_2}")
    ]
)

memory = ConversationBufferWindowMemory(memory_key="history", return_messages=True, k=5)

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    memory=memory
)


response_new = llm_chain.run("hey how are you")

In [9]:
response_new

"I'm just an AI language model, so I don't have feelings or a physical presence. I'm here to help answer your questions to the best of my ability.\n\nRegarding your questions about DDoS attacks and their prevention, here are some additional insights:\n\n1. Keeping the network secure: This involves implementing best security practices, such as using strong, unique passwords, keeping all systems and software up-to-date, using a firewall, and limiting access to sensitive systems. \n\n2. Installing IDS (Intrusion Detection System): An IDS like Snort or Suricata can help detect and alert you to potential DDoS attacks by recognizing patterns in the network traffic. It's important to regularly update the signature database of the IDS to ensure it can recognize the latest attack patterns.\n\n3. Using scanning tools: Regularly scanning your network with tools like Nmap or Nessus can help identify vulnerabilities that might be exploited by DDoS attackers.\n\n4. Running zombie tools: These tools 

In [ ]:
extracted_messages = llm_chain.memory.chat_memory.messages
extracted_messages

In [ ]:
from langchain_core.messages.base import messages_to_dict
ingest_to_db = messages_to_dict(extracted_messages)

In [ ]:
import json
retrieve_from_db = json.loads(json.dumps(ingest_to_db))
retrieve_from_db
